In [1]:
import unidecode
import os
import pandas as pd
import numpy as np
data_path = "./"
texts, labels = [], []
with open(os.path.join(data_path, 'DRUG-AE.rel')) as f:
    for line in f:
        pubmed_id, text = line.strip().split('|')[:2]
        texts.append(text)
        labels.append(1)

with open(os.path.join(data_path, 'ADE-NEG.txt')) as f:
    for line in f:
        pubmed_id, neg = line.strip().split(' ')[:2]
        text = ' '.join(line.strip().split(' ')[2:])
        texts.append(text)
        labels.append(0)

np.random.seed(0)
# Shuffle the data as Keras won't shuffle validation data.

idx = np.random.permutation(len(labels))
labels = np.asarray(labels)[idx]
texts = np.asarray(texts, dtype='str')[idx]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [3]:
texts= texts.tolist()

In [ ]:
from tqdm.auto import tqdm
processed_data = []
for i in tqdm(range(len(texts))):
  text = texts[i]
  label = labels[i]

  encodings = tokenizer(text, padding="max_length", truncation=True, max_length=512,
                      add_special_tokens=True, # Add [CLS] and [SEP] tokens
                      )
  encodings['label'] = label
  encodings['text'] = text
  processed_data.append(encodings)

In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2024
)

In [6]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [7]:
labels = labels.tolist()

In [ ]:
num=list(set(labels))
len(num)

In [ ]:
from transformers import PreTrainedTokenizerFast, PreTrainedModel, AutoTokenizer, AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("./SNOMEDTM")

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f_score, support = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f-score': f_score, 'suppor': support}

In [ ]:
from transformers import TrainingArguments, Trainer
# Define training arguments
training_args = TrainingArguments(
    output_dir="./",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,  # Set the number of epochs
    #save_strategy="steps",  # Save checkpoints every step
    #save_steps=10000,  # Save checkpoints every 1000 steps
    logging_dir="./logs",  # Directory for storing logs
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Start training and optionally resume from checkpoint
trainer.train()

In [ ]:
trainer.evaluate()

In [14]:
import torch
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
def get_prediction(text):
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512,
                             add_special_tokens=True)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1).squeeze().cpu().numpy()

    label = np.argmax(probs, axis=-1)
    return {
        'label': label,
        'probability': max(probs)
    }

In [ ]:
# Lists to store predictions and probabilities
y_pred = []
probability = []

# Iterate over the validation DataFrame
for i in tqdm(range(len(valid_df['text']))):
    pred = get_prediction(valid_df['text'].iloc[i])
    label = pred['label']
    prob = pred['probability']
    y_pred.append(label)
    probability.append(prob)

# Add predictions and probabilities to the DataFrame
valid_df['predictions'] = y_pred
valid_df['probabilities'] = probability

In [17]:
y_prediction = valid_df['predictions']
y_true = valid_df['label']

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold
import matplotlib.pyplot as plt

def results(model_name, y_pred, y_test, classes, multi_class=False, show_cm=False, show_roc=False):
    if multi_class:
        accuracy = round(accuracy_score(y_pred, y_test) * 100, 2)
        precision = round(precision_score(y_pred, y_test, average='macro') * 100, 2)
        recall = round(recall_score(y_pred, y_test, average='macro') * 100, 2)
        f1_scr = round(f1_score(y_pred, y_test, average='macro') * 100, 2)
    else:
        accuracy = round(accuracy_score(y_pred, y_test) * 100, 2)
        precision = round(precision_score(y_pred, y_test) * 100, 2)
        recall = round(recall_score(y_pred, y_test) * 100, 2)
        f1_scr = round(f1_score(y_pred, y_test) * 100, 2)

    # Calculate AUC-ROC score
    auc_roc_score = roc_auc_score(y_test, y_pred)

    # Calculate TP, FP, TN, FN
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    # Classification error
    classification_error = (fp + fn) / (tp + tn + fp + fn)

    print("Accuracy: {}%".format(accuracy))
    print("Precision: {}%".format(precision))
    print("Recall: {}%".format(recall))
    print("F1-Score: {}%".format(f1_scr))
    #print("AUC-ROC Score: {}%".format(auc_roc_score))
    print("True Positive (TP):", tp)
    print("False Positive (FP):", fp)
    print("True Negative (TN):", tn)
    print("False Negative (FN):", fn)
    print("Classification Error: {}%".format(classification_error*100))

    if show_cm:
        print("Classification Report:")
        print(classification_report(y_test, y_pred, target_names=classes))
        print()
        print("Confusion Matrix:")
        fig, ax = plt.subplots(figsize=(7, 5))
        ConfusionMatrixDisplay.from_predictions(y_pred, y_test,
                                                  ax=ax,
                                                  display_labels=classes,
                                                  xticks_rotation='vertical')
        # plt.savefig(f"{save_results}/{model.__class__.__name__}_confusion_matrix.png", bbox_inches='tight')
        plt.show()

    if show_roc:
        # Plot ROC curve
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc_roc_score)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        # plt.savefig(f"{save_results}/{model.__class__.__name__}_auc_roc.png", bbox_inches='tight')
        plt.show()

    return pd.DataFrame([{
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1_scr,
        'AUC-ROC Score': auc_roc_score,
        'True Positive (TP)': tp,
        'False Positive (FP)': fp,
        'True Negative (TN)': tn,
        'False Negative (FN)': fn,
        'Classification Error': round(classification_error*100,2),
    }])

In [ ]:
classes = ['ADE-REL', 'ADE-NO-REL']
res = results("SNOMEDTM_FINETUNING", y_prediction, y_true, classes, multi_class=False, show_cm=True, show_roc=False)
res